In [ ]:
import torch
import torch.nn as nn
from torch.autograd import Variable
from torchvision import models,transforms,utils
import numpy as np
import scipy.misc
import matplotlib.pyplot as plt
from PIL import Image
import json

%matplotlib inline

In [ ]:
transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])
])

In [ ]:
image = Image.open("dog.png")
plt.imshow(image)

In [ ]:
vgg = models.vgg16(pretrained=True)
vgg.classifier[-1]

In [ ]:
CUDA = torch.cuda.is_available()
if CUDA:
    vgg.cuda()

In [ ]:
image = transform(image)

In [ ]:
print(image.shape)

In [ ]:
image = image.unsqueeze(0)

In [ ]:
image=Variable(image)

In [ ]:
output=vgg(image)
#2D -> 1D
output=output.squeeze(0)

In [ ]:
labels = json.load(open('imagenet_class_index.json'))
index = output.max(0)
index = str(index[1][0].item())
label = labels[index][1]
module_list = list(vgg.features.modules())
print(vgg.features)
print(module_list[0])
print(module_list[1])
print(module_list[2])
module_list

In [ ]:
outputs = []
names = []
for layer in module_list[1:]:
    image = layer(image)
    outputs.append(image)
    names.append(str(layer))

In [ ]:
for feature_map in outputs:
    print(feature_map.shape)

In [ ]:
processed = []
for feature_map in outputs:
    feature_map = feature_map.squeeze(0)
    #Convert the 3D Tensor to 2D. Sum the same element of every channel
    gray_scale = torch.sum(feature_map, 0)
    gray_scale = gray_scale/feature_map.shape[0]
    processed.append(gray_scale.data.cpu().numpy())

In [ ]:
fig = plt.figure(figsize = (30,50))

for i in range(len(processed)):
    a = fig.add_subplot(8,4,i+1)
    imgplot = plt.imshow(processed[i])
    plt.axis('off')
    a.set_title(names[i].split('(')[0], fontsize=30)

plt.savefig('feature_maps.jpg', bbox_inches='tight')